#  KoGPT-2로 간단한 Chatbot 구현

## 실험준비

먼저 필요한 자료 가져옵니다.

데이터는 lab_tutorial/data에 담아 두었습니다.

data는 NSMC data와 Chatbot data가 있습니다.

출처: https://github.com/e9t/nsmc, https://github.com/songys/Chatbot_data

In [23]:
!git clone https://github.com/momozzing/lab_tutorial.git

Cloning into 'lab_tutorial'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 17 (delta 3), reused 14 (delta 3), pack-reused 0
Unpacking objects: 100% (17/17), done.


필요한 라이브러리들을 설치합니다. 

pandas, torch, tqdm, tansformers, KoBERT, sentenvepiece

In [24]:
%cd lab_tutorial
!mkdir model_save
!pip install -r requirements.txt
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
%pip install sentencepiece
!pip install pandas==1.1.0                # 버전 충돌때문에 downgrade했습니다. 

/content/lab_tutorial/lab_tutorial
  Using cached pandas-1.3.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.3 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.0
    Uninstalling pandas-1.1.0:
      Successfully uninstalled pandas-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.2 which is incompatible.


  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-cg_8re6g/kobert-tokenizer_e9621a3a971b4270bc5d3e5694461024
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-cg_8re6g/kobert-tokenizer_e9621a3a971b4270bc5d3e5694461024
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(s

## Fine-tuning 

이제 시작입니다. 

필요한 라이브러리들 import합니다.

In [25]:
from argparse import ArgumentParser
import pandas as pd
import os
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelWithLMHead
from torch.optim import AdamW

tokenizer와 model을 define합니다.

생성 모델임으로 AutoModelWithLMHead를 사용합니다. 

Auto는 허깅페이스 내에 올라가있는 모델이 GPT-2모델이면 자동으로 GPT-2모델을 넣어줍니다. (BERT면 자동으로 BERT로)

GPT2LMHeadModel 쓰셔도 됩니다.

In [26]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"                      ## 에러 계속 뜨는것 방지 https://github.com/kakaobrain/pororo/issues/69 참고 

tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")
SPECIAL_TOKENS = {                                       ## special token을 정의합니다. 
    "bos_token": "<bos>",
    "eos_token": "<eos>",
    "pad_token": "<pad>",
    "sep_token": "<seq>"
    }
SPECIAL_TOKENS_VALUES = ["<bos>", "<eos>", "<pad>", "<seq>"]
tokenizer.add_special_tokens(SPECIAL_TOKENS)            ## tokenizer(Vocab)에 special token을 추가합니다. 

model = AutoModelWithLMHead.from_pretrained(
    "skt/kogpt2-base-v2"
).cuda()

model.resize_token_embeddings(len(tokenizer))        ## vocab size를 늘렸으니 임베딩 차원도 같이 늘립니다. 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:902: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Embedding(51204, 768)

데이터 불러옵니다.

컬럼마다 data를 쪼개서 value만 넣어둡니다.

ChatbotData.csv에는 11823 개의 데이터가 있음으로 10000개를 Train data로 사용합니다.

In [46]:
train_data = pd.read_csv("data/ChatbotData.csv", delimiter=",")
print(train_data)
train_data = train_data[:10000]
train_text, train_labels = (
    train_data["Q"].values,
    train_data["A"].values,
)

                             Q                         A  label
0                       12시 땡!                하루가 또 가네요.      0
1                  1지망 학교 떨어졌어                 위로해 드립니다.      0
2                 3박4일 놀러가고 싶다               여행은 언제나 좋죠.      0
3              3박4일 정도 놀러가고 싶다               여행은 언제나 좋죠.      0
4                      PPL 심하네                눈살이 찌푸려지죠.      0
...                        ...                       ...    ...
11818           훔쳐보는 것도 눈치 보임.        티가 나니까 눈치가 보이는 거죠!      2
11819           훔쳐보는 것도 눈치 보임.             훔쳐보는 거 티나나봐요.      2
11820              흑기사 해주는 짝남.                    설렜겠어요.      2
11821  힘든 연애 좋은 연애라는게 무슨 차이일까?  잘 헤어질 수 있는 사이 여부인 거 같아요.      2
11822               힘들어서 결혼할까봐        도피성 결혼은 하지 않길 바라요.      2

[11823 rows x 3 columns]


text의 maxlenght를 찾습니다.

In [47]:
len_check = []
for i in range(len(train_data[:10000])):
  len_check.append(len(train_data["Q"][i])+len(train_data['A'][i]))
print(max(len_check))

79


DataLoader에 넣기 위해 dataset을 정의해줍니다.

User turn에 <usr\> , system turn에 <sys\>, 끝에는 끝을 알리는 <eos\> 토큰들을 넣습니다

In [ ]:
dataset = [
    {"data": "<usr>" + t + "<sys>" + l + str(tokenizer.eos_token), "label": l}
    for t, l in zip(train_text, train_labels)
]

dataloader를 정의합니다. 

참고 https://subinium.github.io/pytorch-dataloader/

In [ ]:
train_loader = DataLoader(
    dataset,
    batch_size=64,
    num_workers=2,
    drop_last=True,
    pin_memory=True,
)

validation data도 똑같이 진행합니다. 

validation data은 ChatbotData.csv에 11823 개의 데이터가 있음으로 1,823개를 Train data로 사용합니다.

In [28]:
eval_data = pd.read_csv("data/ChatbotData.csv", delimiter=",")
eval_data = eval_data[10000:]
eval_text, eval_labels = (
    eval_data["Q"].values,
    eval_data["A"].values,
)
dataset = [
    {"data": "<usr>" + t + "<sys>" + l + str(tokenizer.eos_token), "label": l}
    for t, l in zip(eval_text, eval_labels)
]
eval_loader = DataLoader(
    dataset,
    batch_size=64,
    num_workers=2,
    drop_last=True,
    pin_memory=True,
)

optimizer는 ADAMW를 사용합니다.

In [29]:
optimizer = AdamW(params=model.parameters(),
    lr=3e-5, weight_decay=3e-7
)

train, validation step입니다.

GPT-2의 Casual Attention은 LMHead안에 정의되어있습니다. 
Auto Regressive loss도 잘 정의되어 있습니다. 

참고: https://github.com/huggingface/transformers/blob/v4.15.0/src/transformers/models/gpt2/modeling_gpt2.py#L943

In [33]:
epochs = 10
for epoch in range(epochs):
    model.train()
    for train in tqdm(train_loader):
        optimizer.zero_grad()
        text, label = train["data"], train["label"]
        text_tokens = tokenizer(
            text,
            return_tensors="pt",
            max_length=80,
            truncation=True,
            padding=True,
        )

        input_ids = text_tokens.input_ids.cuda()
        # print(input_ids)
        # print(tokenizer.convert_ids_to_tokens(input_ids[0]))         ## 토크나이저가 어떻게 자르고 어떻게 구성되있는지 보기. 
        attention_mask = text_tokens.attention_mask.cuda()            ## <pad>토큰들은 자동으로 masking되어서 실제 단어만 들어있는것만 학습 진행. 

        output = model.forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=input_ids,
        )

        loss = output.loss
        loss.backward()        
        optimizer.step()

    print({"loss": loss.item()})

    with torch.no_grad():
        model.eval()
        for eval in tqdm(eval_loader):
            eval_text, eval_label = eval["data"], eval["label"]
            eval_text_tokens = tokenizer(
                eval_text,
                return_tensors="pt",
                max_length=80,
                truncation=True,
                padding=True,
            )

            input_ids = eval_text_tokens.input_ids.cuda()
            attention_mask = eval_text_tokens.attention_mask.cuda()

            eval_out = model.forward(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=input_ids,
            )
    
            eval_loss = eval_out.loss

        print({"eval_loss": eval_loss.item()})          
        print({"epoch": epoch+1})
        torch.save(model.state_dict(), f"model_save/GPT-2_fintuing-{epoch+1}.pt")

100%|██████████| 156/156 [01:13<00:00,  2.11it/s]


{'loss': 2.179687261581421}


100%|██████████| 28/28 [00:04<00:00,  6.00it/s]


{'eval_loss': 2.319176435470581}
{'epoch': 1}


100%|██████████| 156/156 [01:13<00:00,  2.12it/s]


...

{'eval_loss': 3.0584371089935303}
{'epoch': 29}


100%|██████████| 156/156 [01:12<00:00,  2.16it/s]


{'loss': 0.49904102087020874}


100%|██████████| 28/28 [00:04<00:00,  6.11it/s]


{'eval_loss': 3.11588716506958}
{'epoch': 30}


## Interactive

In [ ]:
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer

In [34]:
ckpt_name = "model_save/GPT-2_fintuing-10.pt"                       ## check point save file 위치 
model = AutoModelWithLMHead.from_pretrained("skt/kogpt2-base-v2")
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")

SPECIAL_TOKENS = {
    "bos_token": "<bos>",
    "eos_token": "<eos>",
    "pad_token": "<pad>",
    "sep_token": "<seq>"
    }
SPECIAL_TOKENS_VALUES = ["<bos>", "<eos>", "<pad>", "<seq>"]
tokenizer.add_special_tokens(SPECIAL_TOKENS)
model.resize_token_embeddings(len(tokenizer)) 

model.load_state_dict(torch.load(ckpt_name, map_location="cpu"))          ## save point load
model.cuda()

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:902: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51204, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


참고

https://huggingface.co/docs/transformers/v4.15.0/en/internal/generation_utils

https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/02_how_to_generate.ipynb

https://github.com/huggingface/transformers/blob/v4.15.0/src/transformers/generation_utils.py#L741

In [44]:
with torch.no_grad():
    while True:
        t = input("\nUser: ")
        tokens = tokenizer(
            "<usr>" + t,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=80
        )

        input_ids = tokens.input_ids.cuda()
        attention_mask = tokens.attention_mask.cuda()
        sample_output = model.generate(
            input_ids, 
            max_length=80, 
            num_beams=5, 
            early_stopping=True
        )
        gen = sample_output[0]
        print("System: " + tokenizer.decode(gen[len(input_ids[0]):-1], skip_special_tokens=True))


User: 배고파요


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1838: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


System: 얼른 맛난 음식 드세요.

User: 피곤해요
System: 충전하는 시간 그 자체로 소중합니다.

User: 집에가고싶어요
System: 집이 최고죠.

User: 엄마보고싶어요
System: 연락드려보세요.

User: 피곤해요
System: 충전하는 시간 그 자체로 소중합니다.

User: 어디살아요?
System: 이름, 직업 등 말고 온전히 자신으로 사는 거죠.

User: 여행가고싶어요
System: 계획을 세워보세요.

User: 어이없어요
System: 그냥 잊어버리세요.


KeyboardInterrupt: ignored